In [1]:
# pip install fredapi  # https://github.com/mortada/fredapi

In [96]:
import pandas as pd
from fredapi import Fred
fred = Fred(api_key="8bde0229a1293f24e1266a721d90a8d0") # get api key here https://research.stlouisfed.org/docs/api/api_key.html
pd.options.display.max_colwidth = 100

In [112]:
def colgenerator(seriesname, colname):
    '''ONLY FOR CPI VARIABLES'''
    data = fred.get_series(seriesname)
    data = pd.DataFrame(data)
    data = data.rename(columns = {0: "foo"})
    data = data.loc[(data.index >= "1979-01-01") & (data.index <= "2019-12-01")]
    data["year"] = data.index.year
    
    resultraw = data.groupby("year").mean()
    resultreindexed = (resultraw / resultraw.loc[1979, "foo"]) * 100
    resultreindexed = resultreindexed.rename(columns = {"foo": colname})
    
    data["month"] = data.index.month
    data.loc[data["month"].between(1, 3), "quarter"] = 1
    data.loc[data["month"].between(4, 6), "quarter"] = 2
    data.loc[data["month"].between(7, 9), "quarter"] = 3
    data.loc[data["month"].between(10, 12), "quarter"] = 4
    data = data.drop(columns = ["month"])
    
    qresultraw = data.groupby(["year", "quarter"]).mean()
    qresultreindexed = (qresultraw / qresultraw.loc[(1979, 1), "foo"]) * 100
    qresultreindexed = qresultreindexed.rename(columns = {"foo": colname})
    
    return resultreindexed, qresultreindexed
    

In [113]:
cpidata = colgenerator("CPIAUCSL", "cpi") #seasonally adjusted
tcpidata = cpidata[0]
qcpidata = cpidata[1]

In [114]:
tcpidata = round(tcpidata, 3)
foo = pd.DataFrame(pd.Series(tcpidata.to_dict()["cpi"]))
foo = foo.rename(columns = {0: "cpi"})
foo["code"] = "   replace cpi = " + foo["cpi"].astype(str) + " if year == " + foo.index.astype(str)
foo = foo["code"]

In [115]:
print(foo.to_string(index=False))

      replace cpi = 100.0 if year == 1979
    replace cpi = 113.502 if year == 1980
    replace cpi = 125.281 if year == 1981
    replace cpi = 132.997 if year == 1982
    replace cpi = 137.199 if year == 1983
    replace cpi = 143.192 if year == 1984
    replace cpi = 148.243 if year == 1985
    replace cpi = 151.125 if year == 1986
    replace cpi = 156.533 if year == 1987
    replace cpi = 162.951 if year == 1988
    replace cpi = 170.758 if year == 1989
    replace cpi = 180.011 if year == 1990
      replace cpi = 187.6 if year == 1991
    replace cpi = 193.307 if year == 1992
    replace cpi = 199.047 if year == 1993
    replace cpi = 204.214 if year == 1994
    replace cpi = 209.943 if year == 1995
    replace cpi = 216.108 if year == 1996
     replace cpi = 221.16 if year == 1997
    replace cpi = 224.581 if year == 1998
    replace cpi = 229.506 if year == 1999
    replace cpi = 237.233 if year == 2000
    replace cpi = 243.915 if year == 2001
    replace cpi = 247.807 if year 

In [108]:
qcpidata = round(qcpidata, 3)
foo = pd.DataFrame(pd.Series(qcpidata.to_dict()["cpi"]))
foo = foo.rename(columns = {0: "cpi"})
foo = foo.reset_index().rename(columns = {"level_0": "year", "level_1": "quarter"})
foo["code"] = "   replace qcpi = " + foo["cpi"].astype(str) + " if year == " + foo.year.astype(str) + " & quarter == " + foo.quarter.astype(int).astype(str)
foo = foo["code"]

In [109]:
print(foo.to_string(index=False))

      replace qcpi = 100.0 if year == 1979 & quarter == 1
    replace qcpi = 103.179 if year == 1979 & quarter == 2
    replace qcpi = 106.503 if year == 1979 & quarter == 3
    replace qcpi = 109.875 if year == 1979 & quarter == 4
     replace qcpi = 114.21 if year == 1980 & quarter == 1
    replace qcpi = 118.064 if year == 1980 & quarter == 2
    replace qcpi = 120.279 if year == 1980 & quarter == 3
    replace qcpi = 123.651 if year == 1980 & quarter == 4
    replace qcpi = 127.071 if year == 1981 & quarter == 1
    replace qcpi = 129.721 if year == 1981 & quarter == 2
    replace qcpi = 133.333 if year == 1981 & quarter == 3
    replace qcpi = 135.501 if year == 1981 & quarter == 4
    replace qcpi = 136.705 if year == 1982 & quarter == 1
     replace qcpi = 138.68 if year == 1982 & quarter == 2
    replace qcpi = 141.089 if year == 1982 & quarter == 3
    replace qcpi = 141.522 if year == 1982 & quarter == 4
    replace qcpi = 141.618 if year == 1983 & quarter == 1
    replace qc

In [97]:
def colgenerator(stateabb, colname):
    '''ONLY FOR STATE UNEMP VARIABLES'''
    seriesname = stateabb + "UR"
    data = fred.get_series(seriesname)
    data = pd.DataFrame(data)
    data = data.rename(columns = {0: colname})
    data = data.loc[(data.index >= "1979-01-01") & (data.index <= "2019-12-01")]
    data["year"] = data.index.year    
    data["month"] = data.index.month
    data.loc[data["month"].between(1, 3), "quarter"] = 1
    data.loc[data["month"].between(4, 6), "quarter"] = 2
    data.loc[data["month"].between(7, 9), "quarter"] = 3
    data.loc[data["month"].between(10, 12), "quarter"] = 4
    data = data.reset_index()
    data = data.drop(columns = "index")
    data["stateabb"] = stateabb
    return data

In [98]:
statedata = pd.read_csv("/Users/amedeusdsouza/econ499/code/state_cps_fips_abb_name.csv")
statedata = statedata[statedata["state"] < 500]

In [99]:
unempdata = pd.DataFrame()

for i in statedata.index:
    foo = statedata.loc[i, "stateabb"]
    data = colgenerator(foo, "unemp")
    unempdata = unempdata.append(data)

In [100]:
unempdata = unempdata.merge(statedata, on = "stateabb")
unempdata = unempdata.drop(columns = ["stfips", "state_name", "stateabb", "quarter"], axis = 1)
unempdata = unempdata.rename(columns = {"month": "cmonth"})

In [102]:
unempdata.to_stata("/Users/amedeusdsouza/Desktop/econ499data/morg/clean_nber_morg_output/unemp.dta")